In [1]:
import pandas as pd
import numpy as np
import re #정규식 지원 모듈
import warnings
warnings.filterwarnings(action='ignore')
import nltk

In [2]:
data = pd.read_csv('movie_dataset/train.csv')

data.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [3]:
data.isnull().sum() #결측치 없음

id          0
document    0
label       0
dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data)
train.reset_index(inplace=True)
val.reset_index(inplace=True)

In [5]:
train

,index,id,document,label
0,1813,1814,상당히 어린시절에 봤던 영환데 아직도 여운이 남습니다.,1
1,3696,3697,"최고의영화,말이필요없다..눈물,감동..재미..100%",1
2,2451,2452,어줍잫은 해피엔딩. 시간이 소중하다면 좀 보다가 치울 것.,0
3,667,668,이걸 재밋다고 하는거 보면 어휴..,0
4,2001,2002,두배우의 연기력 굿! 블록버스터에 찌들어서 다양성 영,1
...,...,...,...,...
3745,4689,4690,아진차 한회한회를 거듭할수록 너무 재밋다는!미치겟음ㅠ,1
3746,2361,2362,전편의 제목만 빌려와서 시리즈라고 하기엔 무리가 있는 듣보잡인듯,0
3747,96,97,쩌는 시나리오 쩌는 연기 환상의 소피아 결론은 제이미 벨 이 축복받은 놈,1
3748,3489,3490,정말 완벽하게 지루한 시간을 보냈다.,0


In [6]:
val

,index,id,document,label
0,701,702,4.44 맞추기위해 왔습니다~여러분~ 부릉부릉~,0
1,2407,2408,진짜 별볼일없음 여주만 욕하게된다,0
2,4033,4034,일등일드라나나나ㅏ나나아아ㅏㅁ아마,0
3,1626,1627,이런걸볼때마다 감독은 개나소나할수 있는것같다,0
4,744,745,진짜 영애언니 시즌1때보다 살진짜 많이빠지셨고 너무 재밌게잘보고있어요,1
...,...,...,...,...
1245,4308,4309,아이 어린 모두에게 감동을 줄 수 있는 영화,1
1246,2442,2443,호스텔 데드캠프 보다 못하고 마지막은 무슨뜻임??,0
1247,4435,4436,이게 대체 왜 8점대인지 이해할 수 없음.,0
1248,3714,3715,장진영팬이라 어쩔수 없이 봤는데 정말 아니다,0


In [9]:
#품사태깅 후 임의로 품사를 가져와 긍/부정을 판단할 수 있도록 데이터 전처리
from konlpy.tag import Okt

okt = Okt()
print(okt.pos('오늘도 열심히 재미있는')) #test

[('오늘', 'Noun'), ('도', 'Josa'), ('열심히', 'Adverb'), ('재미있는', 'Adjective')]


In [10]:
main_pos = []
for sentence in train['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
train['main_pos'] = pd.DataFrame(main_pos)
train.head()
    

,index,id,document,label,main_pos
0,1813,1814,상당히 어린시절에 봤던 영환데 아직도 여운이 남습니다.,1,상당히 어린시절 봤던 영환 데 아직도 여운 남습니다
1,3696,3697,"최고의영화,말이필요없다..눈물,감동..재미..100%",1,최고 영화 말 필요없다 눈물 감동 재미
2,2451,2452,어줍잫은 해피엔딩. 시간이 소중하다면 좀 보다가 치울 것.,0,어줍잫 해피엔딩 시간 소중하다면 좀 보다가 치울 것
3,667,668,이걸 재밋다고 하는거 보면 어휴..,0,걸 재밋다 하는거 보면
4,2001,2002,두배우의 연기력 굿! 블록버스터에 찌들어서 다양성 영,1,배우 연기력 굿 블록버스터 찌들어서 다양성 영


In [11]:
#CountVectorzier 사용
X_train = train.main_pos
y_train = train.label

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier()
model.fit(X_train_vec, y_train)

RandomForestClassifier()